In [3]:
import json
import spacy
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

In [ ]:
COLLOQUIAL_PEOPLE_AWARDS = ['cecil b. demille award', 'best actress in a motion picture - drama', 'best actor in a motion picture - drama', 'best actress in a motion picture - comedy or musical', 'best actor in a motion picture - comedy or musical', 'best supporting actress in a motion picture', 'best supporting actor in a motion picture', 'best director', 'best actress in a tv series, drama', 'best actor in a tv series, drama', 'best actress in a tv comedy or musical', 'best actor in a tv comedy or musical', 'best actress in a mini-series or tv movie', 'best actor in a mini-series or tv movie', 'best supporting actress (tv)', 'best supporting actor (tv)', 'best screenplay']

In [ ]:
def find_nominee(award_name):
      file_ = open('/content/drive/MyDrive/NLP/gg2013.json')
      data = json.load(file_)
      nominee_corpus = {}
      #nlp = spacy.load("en_core_web_sm")
      #key words to identfy tweet potential to be a nominee
      nominee_keywords = 'nominee|nominated|nomination|nom|deserved|deserve|deserving|robbed|beat'
      won_keywords = 'won|win|wins|winning'+'|'+award_name+' over'
      lost_keywords = 'lose|lost|loses'
      negaive_keywords = 'not nominated'

      #tweet checking
      for tweet_text in data:
        text = tweet_text['text']
        if'rt @' in text.lower():
          continue
        #removing hashtags #
        text = re.sub("#([A-Z])*([a-z])*","", text)
        #removing punctuation symbols
        #text = re.sub(r'[^\w\s]','',text)
        #text = text.lower()
        if award_name in text.lower():
          if re.search(won_keywords,text):
            #tweet = text[:re.search(won_keywords,text).start()]
            tweet = text
            person_ents = return_persons(tweet)
            if len(person_ents) == 0:
              continue
            for person in person_ents:
              if person in nominee_corpus.keys():
                nominee_corpus[person] += 20
              else:
                nominee_corpus[person] = 20
          elif re.search(lost_keywords,text):
            #tweet = text[:re.search(lost_keywords,text).start()]
            tweet = text
            person_ents = return_persons(tweet)
            if len(person_ents) == 0:
              continue
            for person in person_ents:
              if person in nominee_corpus.keys():
                nominee_corpus[person] += 20
              else:
                nominee_corpus[person] = 20
          elif re.search(nominee_keywords,text) and re.search(negaive_keywords,text):
            continue
          elif re.search(nominee_keywords, text):
            #tweet = text[:re.search(nominee_keywords,text).start()]
            tweet = text
            person_ents = return_persons(tweet)
            if len(person_ents) == 0:
              continue
            for person in person_ents:
              if person in nominee_corpus.keys():
                nominee_corpus[person] += 10
              else:
                nominee_corpus[person] = 10
          else:
            person_ents = return_persons(text)
            if len(person_ents) == 0:
              continue
            for person in person_ents:
              if person in nominee_corpus.keys():
                nominee_corpus[person] += 20
              else:
                nominee_corpus[person] = 20
      nominee_corpus = sorted(nominee_corpus.items(),key = lambda x:-x[1])
      #for thing in nominee_corpus:
      #  print (thing)
      return nominee_corpus



In [22]:
text = 'Argo man, what a film. Ben was snubbed for best director at the oscars and Steven did not even win. ridiculous  #goldenglobes'

def return_persons_2_tokens(text):
  tokens = word_tokenize(text)
  pos = pos_tag(tokens)
  sent = ne_chunk(pos, binary=False)

  array = []

  people = []

  entity = ""
  for tree in sent.subtrees(filter=lambda t: t.label() == "PERSON"):
    for leaf in tree.leaves():
      array.append(leaf[0])
    if len(array) == 2:
      for part in array:
        entity += part + " "
      if entity[:-1] not in people:
        people.append(entity[:-1])
      entity = ""
      array = []
  return people

def return_persons_1_tokens(text):
  tokens = word_tokenize(text)
  pos = pos_tag(tokens)
  sent = ne_chunk(pos, binary=False)

  array = []

  people = []

  entity = ""
  for tree in sent.subtrees(filter=lambda t: t.label() == "PERSON"):
    print(tree)
    for leaf in tree.leaves():
      print(leaf)
      array.append(leaf[0])
    print(len(array))
    if len(array) <2:
      for part in array:
        entity += part + " "
      if entity[:-1] not in people:
        people.append(entity[:-1])
    entity = ""
    array = []
  return people

print(return_persons_1_tokens(text))

(PERSON Ben/NNP)
('Ben', 'NNP')
1
(PERSON Steven/NNP)
('Steven', 'NNP')
1
['Ben', 'Steven']


In [ ]:
def really_exists(name):
  ia = IMDb()
  people = ia.search_person(name)
  for person in people:
   if person['name']==name or person['name'] in name or name in person['name']:
      return person['name']
  return None

In [ ]:
if __name__ == '__main__':
    #find_award_names()
    #clean_json()
    #nominee_texts()
    for award in COLLOQUIAL_PEOPLE_AWARDS:
      print('------')
      print(award)
      nominee_corpus = find_nominee(award)
      count=0
      for nominee in nominee_corpus:
        exists = really_exists(nominee[0])
        if (exists):
          print(exists,nominee[1])
          count+=1
        if count==5:
          break      
      print('------')

------
cecil b. demille award
Jodie Foster 530
Cecil B. DeMille 280
Robert Downey Jr. 40
Jodi Foster 40
Selena Gomez 20
------
------
best actress in a motion picture - drama
Jessica Chastain 260
Naomi Watts 20
------
------
best actor in a motion picture - drama
Daniel Lewis 270
------
------
best actress in a motion picture - comedy or musical
Jennifer Lawrence 180
------
------
best actor in a motion picture - comedy or musical
Les Miserables 220
Hugh Jackman 200
------
------
best supporting actress in a motion picture
Anne Hathaway 1000
Les Miserables 780
Les Mize 20
Anne Globe 20
------
------
best supporting actor in a motion picture
Christoph Waltz 280
------
------
best director
Ben Affleck 2850
Best Picture System 160
Quentin Tarantino 60
Ang Lee 40
Jennifer Garner 40
------
------
best actress in a tv series, drama
Claire Danes 140
------
------
best actor in a tv series, drama
Damian Lewis 120
------
------
best actress in a tv comedy or musical
Lena Dunham 40
------
------